In [ ]:
import os
import pandas as pd
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

# Navigate to CS 230 Project folder. This might have a different path depending on user
os.chdir("/content/drive/MyDrive/Classes/CS 230 Project/Data")

Mounted at /content/drive


In [ ]:
def add_eia_file(df, path, col_name):
    """Adds data at `path` to the column `col_name` in `df`"""
    if df is not None:
        new_df = pd.read_csv(path, header=4, names=["DateTime", col_name])
        # Removes the -7:00 UTC
        if col_name is not "hydro_MWh" and col_name is not "demand_MWh":
            new_df["DateTime"] = new_df["DateTime"].apply(lambda x: str(x)[:-6])
        new_df["DateTime"] = pd.to_datetime(new_df["DateTime"])
        new_df.set_index("DateTime", inplace=True)
        df = df.join(new_df)
    else:
        df = pd.read_csv(path, header=4, names=["DateTime", col_name])
        # Removes the -7:00 UTC
        try:
            df["DateTime"] = df["DateTime"].apply(lambda x: x.replace("-07:00",""))
        except:
            pass
        df["DateTime"] = pd.to_datetime(df["DateTime"])
        df.set_index("DateTime", inplace=True)

    return df

In [ ]:
def parse_data(paths, var_names, weather_data=[]):
    df = None
    i = 0
    for path in paths:
        df = add_eia_file(df, path, var_names[i])
        i +=1
            
    for station in weather_data:
        station_name = station.split("_")[1]
        new_df = pd.read_csv(station)
        new_df["DateTime"] =  new_df.apply(lambda x: datetime.strptime(x["Date"], "%m/%d/%Y").replace(hour=int(x["Hour (PST)"] / 100) - 1), axis=1)
        new_df["DateTime"] = pd.to_datetime(new_df["DateTime"])
        new_df.set_index("DateTime", inplace=True)
        col_list = []
        for col in ["ETo (in)", "Precip (in)", "Rel Hum (%)", "Dew Point (F)", "Wind Speed (mph)", "Soil Temp (F)", 
                    "Sol Rad (Ly/day)", "Vap Pres (mBars)", "Wind Dir (0-360)"]:
            new_df[station_name + " " + col] = new_df[col]
            col_list.append(station_name + " " + col)
        new_df.drop(new_df.columns.difference(col_list), axis=1, inplace=True)
        df = df.join(new_df)

    return df

In [6]:
eia_paths = [
    "Net_generation_from_coal_for_CAISO_hourly_-_local_time_07.01.18-04.27.22.csv",
    "Net_generation_from_hydro_for_CAISO_hourly_-_local_time_07.01.18-04.27.22.csv",
    "Net_generation_from_natural_gas_for_CAISO_hourly_-_local_time_07.01.18-04.27.22.csv",
    "Net_generation_from_nuclear_for_CAISO_hourly_-_local_time_07.01.18-04.27.22.csv",
    "Net_generation_from_other_for_CAISO_hourly_-_local_time_07.01.18-04.27.22.csv",
    "Net_generation_from_petroleum_for_CAISO_hourly_-_local_time_07.01.18-04.27.22.csv",
    "Net_generation_from_solar_for_CAISO_hourly_-_local_time_07.01.18-04.27.22.csv",
    "Net_generation_from_wind_for_CAISO_hourly_-_local_time_07.01.18-04.27.22.csv", 
    "Demand_for_CAISO_hourly_-_local_time_07.01.15-04.27.22.csv",
    "Total_interchange_for_CAISO_hourly_-_local_time_07.01.15-04.27.22.csv",         
]
var_names = [
    "coal_MWh",
    "hydro_MWh",
    "ng_MWh",
    "nuclear_MWh",
    "other_MWh",
    "petrol_MWh",
    "solar_MWh",
    "wind_MWh",
    "demand_MWh",
    "interchange_MWh"
]
# select weather data based on renewable locations 
# wind: https://www.calwea.org/fast-facts 
# solar: https://www.solarfeeds.com/mag/solar-farms-in-the-usa/
weather_paths = [
    # "HourlyWeather_Gilroy_2018_2021.csv",
    # "HourlyWeather_Pleasanton_2018_2021.csv",
    "HourlyWeather_ArvinEdison_2018_2021.csv",
    "HourlyWeather_TwitchellIsland_2018_2021.csv",
    "HourlyWeather_Indio_2018_2021.csv",
    "HourlyWeather_Shasta_2018_2021.csv",
    "HourlyWeather_CadizValley_2018_2021.csv",
    "HourlyWeather_Seeley_2018_2021.csv",
    # "HourlyWeather_PaloVerde_2018_2021.csv",
    # "HourlyWeather_SantaMaria_2018_2021.csv"
]
df = parse_data(eia_paths, var_names, weather_data=weather_paths)
df.to_csv("merged_data_v2.csv")